In [1]:
import numpy as np
import pandas as pd
import os
print(os.listdir("E:/data_analytics/quora/"))

['embeddings', 'sample_submission.csv', 'test.csv', 'train.csv']


In [2]:
import nltk
from nltk.corpus import stopwords

In [9]:
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB

In [10]:
train_data = pd.read_csv('E:/data_analytics/quora/train.csv')
test_data = pd.read_csv('E:/data_analytics/quora/test.csv')

In [11]:
train_data.shape

(1306122, 3)

In [13]:
train_data.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [16]:
train_data['target'].value_counts()

Index(['qid', 'question_text', 'target'], dtype='object')

In [17]:
train_data.columns

Index(['qid', 'question_text', 'target'], dtype='object')

In [19]:
#feature extraction

train_text = train_data['question_text']
test_text = test_data['question_text']
train_target = train_data['target']
all_text = train_text.append(test_text)

In [21]:
#fit and transform

tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(all_text)

count_vectorizer = CountVectorizer()
count_vectorizer.fit(all_text)

train_text_features_cv = count_vectorizer.transform(train_text)
test_text_features_cv = count_vectorizer.transform(test_text)

train_text_features_tf = tfidf_vectorizer.transform(train_text)
test_text_features_tf = tfidf_vectorizer.transform(test_text)

In [30]:
kfold = KFold(n_splits = 5, shuffle = True, random_state = 2018)
test_preds1 = 0
oof_preds1 = np.zeros([train_data.shape[0],])
test_preds2 = 0
oof_preds2 = np.zeros([train_data.shape[0],])


for i, (train_idx, valid_idx) in enumerate(kfold.split(train_data)):
    x_train, x_valid = train_text_features_cv[train_idx,:],train_text_features_cv[valid_idx,:]
    y_train, y_valid = train_target[train_idx], train_target[valid_idx]
    classifier1 = MultinomialNB()
    classifier2 = BernoulliNB()
    print('fitting.......')
    classifier1.fit(x_train,y_train)
    classifier2.fit(x_train,y_train)
    print('predicting......')
    print('\n')
    
    oof_preds1[valid_idx] = classifier1.predict_proba(x_valid)[:,1]
    test_preds1 += 0.2*classifier1.predict_proba(test_text_features_cv)[:,1]
    oof_preds2[valid_idx] = classifier2.predict_proba(x_valid)[:,1]
    test_preds2 += 0.2*classifier2.predict_proba(test_text_features_cv)[:,1]

fitting.......
predicting......


fitting.......
predicting......


fitting.......
predicting......


fitting.......
predicting......


fitting.......
predicting......




In [31]:
pred_train = (oof_preds1 > .3).astype(np.int)
f1_score(train_target, pred_train)

0.5336924762600438

In [32]:
pred_train = (oof_preds2 > .3).astype(np.int)
f1_score(train_target, pred_train)

0.523509358390958